# OPENCV

In [ ]:
import numpy as np
import cv2
  
# Open the image.
img = cv2.imread('data\\image1.jpg')
  
# Load the mask.
mask = cv2.imread('data\\mask1.jpg', 0)
  
# Inpaint.
dst = cv2.inpaint(img, mask, 3, cv2.INPAINT_NS)
# dst = cv2.inpaint(img, mask, 3, cv2.INPAINT_TELEA)
  
# Write the output.
cv2.imwrite('opencv_TELEA_NS.png', dst)

# DEEP Learning


In [ ]:
from Deepfill import utils
import argparse
import cv2
import torch
import numpy as np

class Config:
    def __init__(self):
        self.in_channels = 4
        self.out_channels = 3
        self.latent_channels=48
        self.pad_type = 'zero'
        self.activation = 'elu'
        self.norm='none'
        self.init_type = 'xavier'
        self.init_gain=0.02

opt = Config()

generator = utils.create_generator(opt)
generator.load_state_dict(torch.load('Deepfill\\deepfillv2_WGAN_G_epoch40_batchsize4.pth'))
generator.eval()
generator.cuda()


img = cv2.imread('data\\image1.jpg')
mask = cv2.imread('data\\mask1.jpg')[:, :, 0]
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img = torch.from_numpy(img.astype(np.float32) / 255.0).permute(2, 0, 1).contiguous()
mask = torch.from_numpy(mask.astype(np.float32) / 255.0).unsqueeze(0).contiguous()



img = torch.stack([img])
mask = torch.stack([mask])

img = img.cuda()
mask = mask.cuda()

with torch.no_grad():
    first_out, second_out = generator(img, mask)

# forward propagation
first_out_wholeimg = img * (1 - mask) + first_out * mask        # in range [0, 1]
second_out_wholeimg = img * (1 - mask) + second_out * mask     # in range [0, 1]


masked_img = img * (1 - mask) + mask
mask = torch.cat((mask, mask, mask), 1)
img_list = [second_out_wholeimg]
name_list = ['second_out']
utils.save_sample_png(sample_folder = "", sample_name = '%d' % (0+ 1), img_list = img_list, name_list = name_list, pixel_max_cnt = 255)

# DEEP PRIOR

In [ ]:
from Deep_model.deep_prior import deep_prior
import torch
DTYPE = torch.cuda.FloatTensor
INPUT_DEPTH = 32
LR = 0.001 
TRAINING_STEPS = 500
SHOW_STEP = 50
REG_NOISE = 0.03
MAX_DIM = 128
deep_prior(r'data\cat_damaged.png',r'data\cat_mask.png', MAX_DIM, REG_NOISE, INPUT_DEPTH, LR, SHOW_STEP, TRAINING_STEPS)

# Exampler-base


In [1]:
from inpainting import Inpainter
from PIL import Image
from skimage.io import imread, imsave

import numpy as np
image = np.array(Image.open(r'data\cat_damaged.png').convert("RGB"))
# import matplotlib.pyplot as plt
# plt.imshow(image)
mask = np.array(Image.open(r'data\cat_mask.png').convert("L"))/255
inpainter = Inpainter(image, mask, plot_progress=True)
inpainter.inpaint()

Time to find best: 5.531469 seconds
158413 of 160000 completed
Time to find best: 5.423998 seconds
158443 of 160000 completed
Time to find best: 6.461113 seconds
158468 of 160000 completed
Time to find best: 5.800242 seconds
158493 of 160000 completed
Time to find best: 5.737077 seconds
158498 of 160000 completed
Time to find best: 5.554881 seconds
158508 of 160000 completed
Time to find best: 5.231192 seconds
158538 of 160000 completed
Time to find best: 5.687557 seconds
158566 of 160000 completed
Time to find best: 7.532238 seconds
158571 of 160000 completed
Time to find best: 5.623707 seconds
158602 of 160000 completed
Time to find best: 5.589387 seconds
158630 of 160000 completed
Time to find best: 5.197571 seconds
158635 of 160000 completed
Time to find best: 5.742594 seconds
158645 of 160000 completed
Time to find best: 5.389590 seconds
158675 of 160000 completed
Time to find best: 5.504160 seconds
158704 of 160000 completed
Time to find best: 5.484288 seconds
158734 of 160000 co

array([[[118, 105,  96],
        [119, 106,  97],
        [120, 107,  98],
        ...,
        [184, 169, 150],
        [183, 166, 146],
        [182, 165, 145]],

       [[117, 104,  95],
        [118, 105,  96],
        [119, 106,  97],
        ...,
        [182, 165, 147],
        [180, 163, 143],
        [179, 162, 142]],

       [[115, 102,  93],
        [116, 103,  94],
        [117, 104,  95],
        ...,
        [179, 160, 143],
        [177, 159, 139],
        [176, 158, 138]],

       ...,

       [[201, 207, 207],
        [201, 207, 207],
        [201, 207, 207],
        ...,
        [198, 202, 203],
        [200, 204, 205],
        [202, 206, 207]],

       [[197, 203, 203],
        [197, 203, 203],
        [198, 204, 204],
        ...,
        [200, 204, 205],
        [201, 205, 206],
        [201, 205, 206]],

       [[194, 200, 200],
        [194, 200, 200],
        [195, 201, 201],
        ...,
        [200, 204, 205],
        [199, 203, 204],
        [198, 202, 203]]

In [5]:
import numpy as np
import cv2
from glob import glob
width = 400
height = 400

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter('video.avi', fourcc, 15, (width, height))

for idx, image_path in enumerate(glob('results\\*.png')):
    print(image_path)
    img = cv2.imread(f'results\\{idx}.png')
    video.write(img)

cv2.destroyAllWindows()
video.release()

results\0.png
results\1.png
results\10.png
results\11.png
results\12.png
results\13.png
results\14.png
results\15.png
results\16.png
results\17.png
results\18.png
results\19.png
results\2.png
results\20.png
results\21.png
results\22.png
results\23.png
results\24.png
results\25.png
results\26.png
results\27.png
results\28.png
results\29.png
results\3.png
results\30.png
results\31.png
results\32.png
results\33.png
results\34.png
results\35.png
results\36.png
results\37.png
results\38.png
results\39.png
results\4.png
results\40.png
results\41.png
results\42.png
results\43.png
results\44.png
results\45.png
results\46.png
results\47.png
results\48.png
results\49.png
results\5.png
results\50.png
results\51.png
results\52.png
results\53.png
results\54.png
results\55.png
results\56.png
results\57.png
results\58.png
results\59.png
results\6.png
results\60.png
results\61.png
results\62.png
results\63.png
results\64.png
results\65.png
results\66.png
results\67.png
results\68.png
results\69.png
re